In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import no_op, sparse_hog
from menpo.landmark import labeller, ibug_face_66, ibug_face_49
from menpo.visualize import visualize_images, print_dynamic

from menpofit.fitter import noisy_shape_from_shape
from menpofit.sdm import (
    SupervisedDescentFitter, 
    Newton, GaussNewton)
from menpofit.visualize import visualize_fitting_result, plot_ced

## Load training data

In [ ]:
training_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=10, normalise=True):
    i = i.crop_to_landmarks_proportion(0.5)
    i = i.rescale_landmarks_to_diagonal_range(200)
    i.pixels = i.pixels.astype(uint8)
    labeller(i, 'PTS', ibug_face_66)
    training_images.append(i)
    
# for i in mio.import_images('/vol/atlas/databases/helen/trainset/', 
#                            verbose=True, max_images=None, normalise=None):
#     i = i.crop_to_landmarks_proportion(0.5)
#     i = i.rescale_landmarks_to_diagonal_range(200)
#     i.pixels = i.pixels.astype(uint8)
#     labeller(i, 'PTS', ibug_face_66)
#     training_images.append(i)

In [ ]:
visualize_images(training_images)

In [ ]:
i = i.rescale_landmarks_to_diagonal_range(100, 'PTS')

In [ ]:
%%timeit

i.extract_patches(
    PointCloud([[0.0, 0.0]]), 
    patch_size=(17, 17),
    as_single_array=True,
    sample_offsets=PointCloud(np.require(i.indices(),dtype=np.float64)))

## Load test data

In [ ]:
test_images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/testset/', 
                           verbose=True, max_images=None, normalise=None):    
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_66)
    test_images.append(i)

In [ ]:
visualize_images(test_images)

## Train Supervised Fitter

In [ ]:
sd_algorithm_cls = Newton
features = [sparse_hog]
patch_shape = (17, 17)
diagonal = 200
scales = (1, .5)
iterations = 4
n_perturbations = 10

group = 'ibug_face_66'
verbose = True
l = 0

In [ ]:
sd_fitter_batch = SupervisedDescentFitter(
    sd_algorithm_cls=sd_algorithm_cls,
    features=features,
    patch_shape=patch_shape,
    diagonal=diagonal, 
    scales=scales,
    iterations=iterations,
    n_perturbations=n_perturbations)
    
sd_fitter_batch.train(
    training_images,
    group=group,
    verbose=verbose,
    l=l)

In [ ]:
sd_fitter_incremental = SupervisedDescentFitter(
    sd_algorithm_cls=sd_algorithm_cls,
    features=features,
    patch_shape=patch_shape,
    diagonal=diagonal, 
    scales=scales,
    iterations=iterations,
    n_perturbations=n_perturbations)
    
sd_fitter_incremental.train_incrementally(    
    training_images,
    batch_size=300,
    group=group,
    verbose=verbose,
    l=l)

#### Fitting loop

In [ ]:
fitters = [sd_fitter_batch, sd_fitter_incremental]

fitters_results = []

for j, i in enumerate(test_images):
    
    #np.random.seed(j)
    gt_s = i.landmarks['ibug_face_66'].lms
    s = fitters[0].perturb_from_shape(gt_s, noise_std=0.05)

    for fitter in fitters:
        
        fr = fitter.fit(i, s, gt_shape=gt_s) 
        fr.downscale = 0.5
        fitters_results.append(fr)
    
        print 'Image: ', j
        print fr

## Results

In [ ]:
initial_errors = [fr.initial_error() for fr in fitters_results[::2]]

final_errors_1 = [fr.final_error() for fr in fitters_results[::2]]
final_errors_2 = [fr.final_error() for fr in fitters_results[1::2]]

In [ ]:
print 'ini:', np.mean(initial_errors), 'std_1:', np.std(initial_errors), 'median:', np.median(initial_errors) 
print 'mean_1:', np.mean(final_errors_1), 'std_1:', np.std(final_errors_1), 'median:', np.median(final_errors_1) 
print 'mean_2:', np.mean(final_errors_2), 'std_2:', np.std(final_errors_2), 'median:', np.median(final_errors_2) 

In [ ]:
errors = [initial_errors, final_errors_1, final_errors_2]

In [ ]:
plot_ced(errors, legend_entries=['ini', '1', '2'])

In [ ]:
visualize_fitting_result(fitters_results[::2])

In [ ]:
visualize_fitting_result(fitters_results[1::2])

## Timing

In [ ]:
%timeit -n 25 fr = sd_fitter_batch.fit(i, s, gt_shape=gt_s) 
%timeit -n 25 fr = sd_fitter_incremental.fit(i, s, gt_shape=gt_s) 

In [ ]:
import line_profiler
import IPython

ip = IPython.get_ipython()
ip.define_magic('lprun', line_profiler.magic_lprun)

In [ ]:
%lprun -f Newton.run fitter.fit(i, s, gt_shape=gt_s)